<a href="https://colab.research.google.com/github/DavorJordacevic/OpenCV/blob/master/Epipolar_Geometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import numpy as np

In [0]:
# read the left image
image_left = cv2.imread('left.jpg',0)
# read the right image
image_right = cv2.imread('right.jpg',0)

**ORB (Oriented FAST and Rotated BRIEF)**

In [0]:
# Initiate ORB detector
orb = cv2.ORB_create(2000)

# Find the keypoints with ORB
kp1 = orb.detect(image_left,None)
kp2 = orb.detect(image_right,None)

# Compute the descriptors with ORB
kp1, des1 = orb.compute(image_left, kp1)
kp2, des2 = orb.compute(image_right, kp2)

# Convert descriptors to float32
des1 = np.float32(des1)
des2 = np.float32(des2)

In [0]:
# Make a copy of both images
image1_copy= image_left
image2_copy= image_right

# Draw keypoints
image1_copy = cv2.drawKeypoints(image_left,kp1,None,(255,0,0),4)
image2_copy = cv2.drawKeypoints(image_right,kp2,None,(255,0,0),4)

**FLANN MATCHER**

In [0]:
# we need to define some FLANN(Fast Approximate Nearest Neighbor Search Library) parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

In [0]:
# Matching descriptor vectors using FLANN matcher
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

In [0]:
good = []
points1 = []
points2 = []

In [0]:
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        points1.append(kp1[m.queryIdx].pt)
        points2.append(kp2[m.trainIdx].pt)

**FUNDAMENTAL MATRIX**

In [0]:
points1 = np.int32(points1)
points2 = np.int32(points2)
# This is an important step. So here we need to find the fundamental matrix
F, mask = cv2.findFundamentalMat(points1,points2,cv2.FM_LMEDS)

In [0]:
print("Fundamental matrix:\n", F)

In [0]:
# We select only inlier points
points1 = points1[mask.ravel()==1]
points2 = points2[mask.ravel()==1]

**EPIPOLAR LINES**

In [0]:
# Function for drawing lines
def drawlines(image1,lines,points1,points2):
    rows,columns = image1.shape
    image1 = cv2.cvtColor(image1,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,points1,points2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [columns, -(r[2]+r[0]*columns)/r[1] ])
        image1 = cv2.line(image1, (x0,y0), (x1,y1), color,1)
    return image1

In [0]:
# Find epilines corresponding to points in right image (second image) and
# drawing its lines on left image
lines1 = cv2.computeCorrespondEpilines(points1.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
epilines_left = drawlines(image_left,lines1,points1,points2)

In [0]:
# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
lines2 = cv2.computeCorrespondEpilines(points1.reshape(-1,1,2), 1,F)
lines2 = lines2.reshape(-1,3)
epilines_right = drawlines(image_right,lines2,points2,points1)

In [0]:
cv2.imshow("Left image",epilines_left)
cv2.imshow("Right image",epilines_right)

cv2.waitKey(0)
cv2.DestroyAllWindows()